In [1]:
import pandas as pd
import numpy
import xlrd
import re
from zipfile import ZipFile

import xlrd
from zipfile import ZipFile

def open_zip(datafile):
    with ZipFile('{0}'.format(datafile), 'r') as myzip:
        myzip.extractall()   

In [35]:
def process_timesheet(unzipped_datafile, timesheet):
    workbook = xlrd.open_workbook(unzipped_datafile)
    sheet = workbook.sheet_by_name('Timesheet')
    
    #Process first tab of timesheet
    #On the first tab, save cell B3 as the contractor name
    name = sheet.cell_value(2, 1)
    #On the first tab, save cell B8 as the contractor_id
    contractor_id = str(sheet.cell_value(3,4))[:3]
        #On the first tab, save cell D18 as the contractor rate
    payrate = sheet.cell_value(17,3)
    if payrate == "Hourly \nRate":
        payrate = sheet.cell_value(18,3)  #likely a more elegant solution, but brute forcing for now
    date = re.search('\d+[-]\d+[-]\d+', unzipped_datafile).group(0)

    #Process second tab of Excel spreadsheet
    sheet2 = workbook.sheet_by_name('Task Description')
    #On the second tab, starting with cell A6:E6, save each row until you encounter an empty row
    row = 5
    hours = sheet2.cell_value(row,0)
    activity = sheet2.cell_value(row, 1)
    description = sheet2.cell_value(row, 4)
    budget_category = sheet2.cell_value(row,8)
    while hours != '' and activity != "Total":
        cost = payrate * hours
        timesheet = timesheet.append({'Name':name, 'Contractor_ID': contractor_id, 'Payrate':payrate, 'Invoice Date':date, 'Hours':hours, 'Cost': cost, 'Activity':activity, 'Description':description, 'Budget Category': budget_category}, ignore_index=True)
        row += 1
        hours = sheet2.cell_value(row,0)
        activity = sheet2.cell_value(row, 1)
        description = sheet2.cell_value(row, 4)
        budget_category = sheet2.cell_value(row, 8)
    return timesheet, date

In [37]:
import glob

timesheet = pd.DataFrame([],columns=['Name', 'Contractor_ID', 'Invoice Date','Payrate','Hours', 'Cost', 'Activity','Description', 'Budget Category'])

#for file in glob.glob("*.zip"):    # only loops over zip files
#        open_zip(file)
        
#apply process_timesheet to all unzipped Excel files
for file in glob.glob("*.xls*"):
        timesheet, date = process_timesheet(file, timesheet)

In [5]:
def scrub_budget_category(category):
    if re.search('(Standards)', category) is not None:
        category = "Standards Collections"
    if re.search('(Activation)', category) is not None:
        category = "Navigate Activation"
    if re.search('(Maint)', category) is not None:
        category = "Navigate Content - Maintenance"
    if re.search('(New Dev)', category) is not None:
        category = "Navigate Content - New Development"
    if re.search('(New Prod)', category) is not None:
        category = "Navigate Content - New Product Development"
    return category

timesheet['Budget Category'] = timesheet['Budget Category'].apply(scrub_budget_category)

In [40]:
category_totals = timesheet.groupby('Budget Category').sum()

In [39]:
budget = timesheet[['Contractor_ID', 'Budget Category', 'Cost']]
budget_group = budget.groupby(['Contractor_ID', 'Budget Category']).sum().reset_index()